## 第10章 序列的修改、散列和切片

- 用户定义的序列类型
    - 协议和鸭子类型

In [22]:
# 头文件

from array import array
import reprlib
import math
import collections
import numbers

In [4]:
# help(array)

In [5]:
# Demo_10-2

class Vector:
    typecode = 'd' # 转换成浮点数
    def __init__(self, components):
        self._components = array(self.typecode, components)  # “受保护的”实例属性，把Vector的分量保存在一个数组中
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        components = reprlib.repr(self._components) # 使用reprlib.repr（　）函数获取self._components的有限长度表示形式
        components = components[components.find('['): -1]
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

- reprlib.repr():
    - 这个函数用于生成大型结构或递归结构的安全表示形式，它会限制输出字符串的长度，用'...'表示截断的部分
- 调用repr（　）函数的目的是调试，因此绝对不能抛出异常。


In [5]:
# Demo_10-3

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'SPADES DIAMONDS CLUBS HEARTS'.split( )
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getItem__(self, position):
        return self._cards[position]

In [9]:
# 切片原理
# Demo_10-4

class MySeq:
    def __getitem__(self, index):
        return index

obj_s = MySeq()
print(obj_s[1])
print(obj_s[1:4])
print(obj_s[1:4:2])
print(obj_s[1:4:2, 9])  # 如果[]中有逗号，那么__getitem__收到的是元组
print(obj_s[1:4:2, 7:9])  # 元组中甚至可以有多个切片对象

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [10]:
# Demo_10-5
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [11]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [14]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [15]:
slice(-3, None, None).indices(5)

(2, 5, 1)

In [25]:
# Demo_10-6
class Vector:
    typecode = 'd' # 转换成浮点数
    def __init__(self, components):
        self._components = array(self.typecode, components)  # “受保护的”实例属性，把Vector的分量保存在一个数组中
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        components = reprlib.repr(self._components) # 使用reprlib.repr（　）函数获取self._components的有限长度表示形式
        components = components[components.find('['): -1]
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index]) # 调用类的构造方法，使用_components数组的切片构建一个新Vector实例
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msq = '{cls.__name__} indices must be integers'
            raise TypeError(msq.format(cls=cls))

In [26]:
v_7 = Vector(range(7))
print(v_7[-1])
print(v_7[1:4])
print(v_7[-1:])
print(v_7[1,2])

6.0
(1.0, 2.0, 3.0)
(6.0,)


TypeError: Vector indices must be integers

In [31]:
# Demo_10-8
class Vector:
    typecode = 'd' # 转换成浮点数
    def __init__(self, components):
        self._components = array(self.typecode, components)  # “受保护的”实例属性，把Vector的分量保存在一个数组中
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        components = reprlib.repr(self._components) # 使用reprlib.repr（　）函数获取self._components的有限长度表示形式
        components = components[components.find('['): -1]
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index]) # 调用类的构造方法，使用_components数组的切片构建一个新Vector实例
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msq = '{cls.__name__} indices must be integers'
            raise TypeError(msq.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):  # 仅当对象没有指定名称的属性时，Python才会调用那个方法。
        cls = type(self)
        if 1 == len(name):
            pos = cls.shortcut_names.find(name)
            if 0 <= pos <= len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

In [32]:
v_8 = Vector(range(10))
print(v_8.x)
print(v_8.y)
print(v_8.z)
print(v_8.t)

0.0
1.0
2.0
3.0


In [3]:
# Demo_10-10
class Vector:
    typecode = 'd' # 转换成浮点数
    def __init__(self, components):
        self._components = array(self.typecode, components)  # “受保护的”实例属性，把Vector的分量保存在一个数组中
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        components = reprlib.repr(self._components) # 使用reprlib.repr（　）函数获取self._components的有限长度表示形式
        components = components[components.find('['): -1]
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index]) # 调用类的构造方法，使用_components数组的切片构建一个新Vector实例
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msq = '{cls.__name__} indices must be integers'
            raise TypeError(msq.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):  # 仅当对象没有指定名称的属性时，Python才会调用那个方法。
        cls = type(self)
        if 1 == len(name):
            pos = cls.shortcut_names.find(name)
            if 0 <= pos <= len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
    def __setter__(self, name, value):
        cls = type(self)
        if 1 == len(name):
            if name in cls.shortcut_names:
                error = ' readonly attribute {attr_name!r}'
            elif name.islower():
                error = 'can not set attributes "a" to "z" in {cls_name!r}'
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

In [11]:
# Demo_10-11 计算累计异或的3种方式

import functools
import operator

# 第一种方法
n = 0
for i in range(1, 6):
    n ^= i

print(n)

# 第二种方法
n_2 = functools.reduce(lambda a, b: a^b, range(0,6))
print(n_2)

# 第三种方法
n_3 = functools.reduce(operator.xor, range(6))
print(n_3)

1
1
1


In [23]:
# Demo_10-12

from array import array
import reprlib
import math
import functools
import operator

class Vector:
    typecode = 'd' # 转换成浮点数
    def __init__(self, components):
        self._components = array(self.typecode, components)  # “受保护的”实例属性，把Vector的分量保存在一个数组中
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        components = reprlib.repr(self._components) # 使用reprlib.repr（　）函数获取self._components的有限长度表示形式
        components = components[components.find('['): -1]
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    def __eq__(self, other):
        # 比较浪费内存
        # return tuple(self) == tuple(other)
        # 使用zip进行优化
        if len(self) != len(other):
            return False
        for a, b in zip(self, other):
            if a != b:
                return False
        return True
        # 使用all 继续优化
        # return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index]) # 调用类的构造方法，使用_components数组的切片构建一个新Vector实例
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msq = '{cls.__name__} indices must be integers'
            raise TypeError(msq.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):  # 仅当对象没有指定名称的属性时，Python才会调用那个方法。
        cls = type(self)
        if 1 == len(name):
            pos = cls.shortcut_names.find(name)
            if 0 <= pos <= len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))      
    def __setter__(self, name, value):
        cls = type(self)
        if 1 == len(name):
            if name in cls.shortcut_names:
                error = ' readonly attribute {attr_name!r}'
            elif name.islower():
                error = 'can not set attributes "a" to "z" in {cls_name!r}'
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
    def __hash__(self):
        # hashes = (hash(x) for x in self._components)
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)

In [29]:
# Demo_10-15
# zip内置函数的使用示例

print(zip(range(3), 'ABC'))
print(list(zip(range(3), 'ABC')))
print(list(zip(range(3), 'ABC', (1.1, 2.2, 3.3, 4.4))))

from itertools import zip_longest  # 以最长的序列进行拆分
print(list(zip_longest(range(3), 'ABC', (1.1, 2.2, 3.3, 4.4), fillvalue='$$')))

[(0, 'A'), (1, 'B'), (2, 'C')]
[(0, 'A', 1.1), (1, 'B', 2.2), (2, 'C', 3.3)]
[(0, 'A', 1.1), (1, 'B', 2.2), (2, 'C', 3.3), ('$$', '$$', 4.4)]


In [30]:
# Demo_10-16
# 本章的终极代码

from array import array
import reprlib
import math
import functools
import operator
import itertools

class Vector:
    typecode = 'd' # 转换成浮点数
    def __init__(self, components):
        self._components = array(self.typecode, components)  # “受保护的”实例属性，把Vector的分量保存在一个数组中
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        components = reprlib.repr(self._components) # 使用reprlib.repr（　）函数获取self._components的有限长度表示形式
        components = components[components.find('['): -1]
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    def __eq__(self, other):
        # 比较浪费内存
        # return tuple(self) == tuple(other)
        # 使用zip进行优化
        if len(self) != len(other):
            return False
        for a, b in zip(self, other):
            if a != b:
                return False
        return True
        # 使用all 继续优化
        # return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index]) # 调用类的构造方法，使用_components数组的切片构建一个新Vector实例
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msq = '{cls.__name__} indices must be integers'
            raise TypeError(msq.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):  # 仅当对象没有指定名称的属性时，Python才会调用那个方法。
        cls = type(self)
        if 1 == len(name):
            pos = cls.shortcut_names.find(name)
            if 0 <= pos <= len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))      
    def __setter__(self, name, value):
        cls = type(self)
        if 1 == len(name):
            if name in cls.shortcut_names:
                error = ' readonly attribute {attr_name!r}'
            elif name.islower():
                error = 'can not set attributes "a" to "z" in {cls_name!r}'
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
    def __hash__(self):
        # hashes = (hash(x) for x in self._components)
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) -1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_space = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
        